# <center> <img src="../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Big Data** </center>
---
### <center> **Autumn 2025** </center>
---
### <center> **Examples on Machine Learning: Logistic Regression** </center>
---
**Profesor**: Pablo Camarillo Ramirez

## Recreate docker images and spark cluster:

Go to **spark** directory and run:
     
     sh build-images.sh
     docker compose up --scale spark-worker=1 -d

# Create SparkSession

In [ ]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("ML: Logistic Regression") \
    .master("spark://spark-master:7077") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("INFO")

# Optimization (reduce the number of shuffle partitions)
spark.conf.set("spark.sql.shuffle.partitions", "5")

# Collect Data

In [ ]:
from pcamarillor.spark_utils import SparkUtils
# Create a small dataset as a list of tuples
# Format: (label, feature_x1, feature_x2)
data = [
    (1.0, 2.0, 3.0),
    (0.0, 1.0, 2.5),
    (1.0, 3.0, 5.0),
    (0.0, 0.5, 1.0),
    (1.0, 4.0, 6.0)
]

# Define schema for the DataFrame
schema = SparkUtils.generate_schema([("label", "float"), 
                                     ("feature_x1", "float"),
                                     ("feature_x2", "float")])

# Convert list to a DataFrame
df = spark.createDataFrame(data, schema)

### Assemble the features into a single vector column

In [ ]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=["feature_x1", "feature_x2"], outputCol="features")
data_with_features = assembler.transform(df).select("label", "features")                                      

# Data splitting
#### 80% training data and 20% testing data

In [ ]:
train_df, test_df = data_with_features.randomSplit([0.8, 0.2], seed=57)

### Show dataset (for debugging)

In [ ]:
print("Original Dataset")
data_with_features.show()

# Print train dataset
print("train set")
train_df.show()

# Create ML Model

In [ ]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(maxIter=10, regParam=0.01)

# Train ML Model

In [ ]:
lr_model = lr.fit(train_df)

# Print coefficients
print("Coefficients: " + str(lr_model.coefficients))

# Display model summary
training_summary = lr_model.summary

## Predictions

In [ ]:
# Use the trained model to make predictions on the test data
predictions = lr_model.transform(test_df)

# Show predictions
predictions.select("features", "prediction", "probability").show()

# Test ML Model

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(labelCol="label",
                            predictionCol="prediction")

accuracy = evaluator.evaluate(predictions, 
                  {evaluator.metricName: "accuracy"})
print(f"Accuracy: {accuracy}")
precision = evaluator.evaluate(predictions,
                  {evaluator.metricName: "weightedPrecision"})
print(f"Precision: {precision}")
recall = evaluator.evaluate(predictions,
                  {evaluator.metricName: "weightedRecall"})
print(f"Recall: {recall}")
f1 = evaluator.evaluate(predictions,
                {evaluator.metricName: "f1"})
print(f"F1 Score: {f1}")  

# Lab 11: Logistic regression To predict heart disease

# Data collection

In [ ]:
# Define schema for the DataFrame
heart_schema = SparkUtils.generate_schema([
    ("male", "int"), 
    ("age", "int"), 
    ("education", "int"), 
    ("currentSmoker", "int"), 
    ("cigsPerDay", "int"), 
    ("BPMeds", "int"), 
    ("prevalentStroke", "int"), 
    ("prevalentHyp", "int"), 
    ("diabetes", "int"), 
    ("totChol", "int"), 
    ("sysBP", "float"), 
    ("diaBP", "float"), 
    ("BMI", "float"), 
    ("heartRate", "int"), 
    ("glucose", "int"), 
    ("TenYearCHD", "int")])

# Source: https://www.kaggle.com/datasets/dileep070/heart-disease-prediction-using-logistic-regression?resource=download

heart_df = spark.read \
                .option("header", "true") \
                .schema(heart_schema) \
                .csv("/opt/spark/work-dir/data/ml/logistic_regression/framingham.csv")

heart_df.printSchema()

In [ ]:
heart_df.show(2)

# Data Splitting

# Create ML Model

# Train ML Model

# Test ML Model

In [ ]:
sc.stop()